This file is used for calculating the mean and std of the dataset in order to normalize image tensors before training the neural network.

Gain Access to Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Unzip the dataset file

In [ ]:
!unzip -q 'DATASET_ZIP_FILE_PATH'

In [3]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [4]:
import torch
from torch.utils.data import DataLoader,Dataset
from torchvision import datasets
import torchvision.transforms.v2 as transforms

In [ ]:
train_dir = "TRAIN_SET_PATH"
val_dir = "VALIDATION_SET_PATH"
test_dir = "TEST_SET_PATH"

In [6]:
transforms = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()])

train_dataset = datasets.ImageFolder(root=train_dir,transform=transforms)
val_dataset = datasets.ImageFolder(root=val_dir,transform=transforms)
test_dataset = datasets.ImageFolder(root=test_dir,transform=transforms)

dataset = torch.utils.data.ConcatDataset([train_dataset,val_dataset,test_dataset])
data_loader = DataLoader(dataset,batch_size=32,num_workers=0,shuffle=False)

/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


Calculate mean and std based on code from pytorch forums https://discuss.pytorch.org/t/computing-the-mean-and-std-of-dataset/34949/32

In [9]:
mean = 0.0
for images, _ in data_loader:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
mean = mean / len(data_loader.dataset)

var = 0.0
for images, _ in data_loader:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    var += ((images - mean.unsqueeze(1))**2).sum([0,2])
std = torch.sqrt(var / (len(data_loader.dataset)*224*224))

print(mean,std)

tensor([0.5691, 0.4458, 0.3910]) tensor([0.2746, 0.2446, 0.2383])
